In [1]:

import IPython
import pandas as pd
import numpy as np

from datetime import datetime, date
from math import *
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import re
from decimal import Decimal

from os import listdir
from os.path import isfile, join
from datetime import timedelta

In [2]:
# df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']


## Load historical Quote/Base prices

In [81]:

# df_btc = load_candles('BTC_GBP')
# df_eth = load_candles('ETH_GBP')
# df_usdt = load_candles('GBP_USDT')
# df_usdt = pd.DataFrame({'time': df_usdt.time, 'ohl3': df_usdt.ohl3**(-1)})

,time,avg
time,,
BNB_GBP-202009010100,2020-09-01 01:00:00,17.450000
BNB_GBP-202009010101,2020-09-01 01:01:00,17.450000
BNB_GBP-202009010102,2020-09-01 01:02:00,17.450000
BNB_GBP-202009010103,2020-09-01 01:03:00,17.450000
BNB_GBP-202009010104,2020-09-01 01:04:00,17.450000
...,...,...
BNB_GBP-202306170137,2023-06-17 01:37:00,185.800000
BNB_GBP-202306170138,2023-06-17 01:38:00,185.800000
BNB_GBP-202306170139,2023-06-17 01:39:00,185.900000


In [ ]:

df_trades['ntime'] = df_trades.mtime.dt.strftime('%Y%m%d%H%M').astype(int)

## Load Binance data

In [454]:
# Fee is not included in the total

df_binance = pd.concat([
    pd.read_csv(filepath, parse_dates=True ) for filepath in [
        './trading-data/binance/trading-history-2020.csv',
        './trading-data/binance/trading-history-2021.csv',
        './trading-data/binance/trading-history-2022.csv'
    ]
], axis=0)

df_binance.columns = ['date', 'pair', 'side', 'price', 'executed', 'amount', 'fee']
df_binance['time'] = pd.to_datetime(df_binance['date'], utc=True).dt.floor('S').dt.tz_convert('Europe/London')#.dt.floor('s')
df_binance.price = df_binance.price.str.replace(',','').astype('float')


re_pair = re.compile('(?P<base>[a-z]+)(?P<quote>GBP|USDT|ETH|BTC|USDC|DAI|BUSD)', re.IGNORECASE)
re_curr_amout = re.compile('^(?P<amt>[\d\.,]+)(?P<curr>\w+)$')

df_binance[['base', 'quote']] = df_binance.pair.str.extract(re_pair, expand=True)
df_binance[['executed', 'executed_curr']]  = df_binance.executed.str.split(re_curr_amout, expand=True).iloc[:,[1,2]]
df_binance[['amount', 'amount_curr']]  = df_binance.amount.str.split(re_curr_amout, expand=True).iloc[:,[1,2]]
df_binance[['fee', 'fee_curr']]  = df_binance.fee.str.split(re_curr_amout, expand=True).iloc[:,[1,2]]
for nc in ['executed', 'fee', 'amount']:
    df_binance[nc] = pd.to_numeric(df_binance[nc].str.replace(',', ''))
# df_binance['executed', 'fee', 'amount'] = 
# pd.to_numeric(df_binance[['executed', 'fee', 'amount']].values())
df_binance['exchange'] = 'binance'
# df_binance.tail()d
### !!! fee is not included in amount

# Checks
print('Checks', json.dumps({
    '# records': df_binance.shape[0],
    'base.isnull()' : df_binance[df_binance.base.isnull()].shape[0],
    'quote.isnull()' : df_binance[df_binance.quote.isnull()].shape[0],
    'price * executed != amount' : df_binance[(df_binance.price * df_binance.executed - df_binance.amount).abs() > 0.00001].shape[0],
    'executed_curr != base' : df_binance[df_binance.executed_curr != df_binance.base].shape[0],
    'amount_curr != quote' : df_binance[df_binance.amount_curr != df_binance.quote].shape[0],
    'fee_curr != quote & fee_curr != base' : df_binance[(df_binance.fee_curr != df_binance.quote) & (df_binance.fee_curr != df_binance.base)].shape[0],
    'price, amount, executed, fee < 0' : int((df_binance[['price', 'amount', 'executed', 'fee']] < 0).sum().sum())
}, indent=2))
df_binance.rename({'amount': 'total', 'executed': 'size'}, axis=1, inplace=True)
# df_binance.drop(['executed_curr', 'amount_curr', 'pair'], inplace=True, axis=1 )
df_binance = df_binance[['time', 'exchange', 'pair', 'base', 'quote', 'side', 'price', 'size', 'total', 'fee', 'fee_curr']]
df_binance

Checks {
  "# records": 26578,
  "base.isnull()": 0,
  "quote.isnull()": 0,
  "price * executed != amount": 0,
  "executed_curr != base": 0,
  "amount_curr != quote": 0,
  "fee_curr != quote & fee_curr != base": 5122,
  "price, amount, executed, fee < 0": 0
}


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
0,2020-12-30 02:01:29+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.150000,3064.500000,0.0,BTC
1,2020-12-30 02:01:29+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.000003,0.061290,0.0,BTC
2,2020-12-30 02:01:28+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.100000,2043.000000,0.0,BTC
3,2020-12-30 02:01:28+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.634890,12970.802700,0.0,BTC
4,2020-12-30 02:01:28+00:00,binance,BTCGBP,BTC,GBP,BUY,20430.00,0.150000,3064.500000,0.0,BTC
...,...,...,...,...,...,...,...,...,...,...,...
15,2022-09-16 23:13:15+01:00,binance,BTCGBP,BTC,GBP,BUY,17291.62,0.018270,315.917897,0.0,BNB
16,2022-09-11 17:20:07+01:00,binance,BTCUSDT,BTC,USDT,BUY,21694.29,0.025000,542.357250,0.0,BNB
17,2022-09-11 17:20:07+01:00,binance,BTCUSDT,BTC,USDT,BUY,21694.41,0.195990,4251.887416,0.0,BNB
18,2022-09-11 17:20:07+01:00,binance,BTCUSDT,BTC,USDT,BUY,21694.29,0.120000,2603.314800,0.0,BNB


In [58]:
# df_binance[(df_binance.fee_curr != df_binance.quote) & (df_binance.fee != 0) & (df_binance.fee_curr != df_binance.base)].fee_curr.unique()
df_binance[(df_binance.fee_curr == df_binance.quote)]
# df_binance[['fee', 'fee_curr']][df_binance.fee != 0].groupby('fee_curr').sum().round()
# df_binance['date'].dt.floor('S')
# df_binance.quote.unique()
# (df_binance[['price', 'size', 'total', 'fee']] < 0).sum().sum()

,time,ntime,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
6,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.025948,530.63660,0.0,GBP
7,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.000002,0.04090,0.0,GBP
8,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.109506,2239.39770,0.0,GBP
9,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.001285,26.27825,0.0,GBP
10,2020-12-30 00:57:56+00:00,202012300057,binance,BTCGBP,BTC,GBP,SELL,20450.0,0.003635,74.33575,0.0,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...
26109,2021-01-04 21:25:41+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.000006,0.13902,0.0,GBP
26110,2021-01-04 21:25:35+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.861400,19958.63800,0.0,GBP
26111,2021-01-04 21:25:35+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.073700,1707.62900,0.0,GBP
26112,2021-01-04 21:25:35+00:00,202101042125,binance,BTCGBP,BTC,GBP,SELL,23170.0,0.150000,3475.50000,0.0,GBP


In [36]:
20228.62	* 0.113151

2288.88858162

In [26]:
dada = df_binance[(df_binance.fee != 0) & (df_binance.fee_curr == 'ADA')]
dada['ttot'] = dada.size * dada.price
dada

/tmp/ipykernel_4420/2318426777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dada['ttot'] = dada.size * dada.price


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ttot
331,2020-12-26 01:31:12+00:00,binance,ADABTC,ADA,BTC,BUY,0.000006,41770.0,0.263151,41.7700,ADA,0.046777
332,2020-12-26 01:31:12+00:00,binance,ADABTC,ADA,BTC,BUY,0.000006,3933.0,0.024778,3.9330,ADA,0.046777
333,2020-12-26 01:31:12+00:00,binance,ADABTC,ADA,BTC,BUY,0.000006,2038.0,0.012839,2.0380,ADA,0.046777
53,2021-11-21 14:44:00+00:00,binance,ADAUSDT,ADA,USDT,BUY,1.869000,1530.5,2860.504500,1.5305,ADA,13877.325000
54,2021-11-21 14:44:00+00:00,binance,ADAUSDT,ADA,USDT,BUY,1.869000,160.5,299.974500,0.1605,ADA,13877.325000
...,...,...,...,...,...,...,...,...,...,...,...,...
20559,2021-02-06 18:21:12+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,2674.0,1513.484000,2.6740,ADA,4202.550000
20560,2021-02-06 18:21:03+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,25.3,14.319800,0.0253,ADA,4202.550000
20561,2021-02-06 18:21:03+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,444.5,251.587000,0.4445,ADA,4202.550000
20562,2021-02-06 18:21:03+00:00,binance,ADAUSDT,ADA,USDT,BUY,0.566000,25.8,14.602800,0.0258,ADA,4202.550000


In [27]:
0.566000 * 2674.0

1513.484

## Load Coinbase Pro data

In [41]:
df_coinbase = pd.read_csv('./trading-data/coinbase-pro/fills.csv', parse_dates=True) 
df_coinbase.rename({'product': 'pair', 
                    'created at': 'date',
                    }, inplace=True, axis='columns')
df_coinbase.pair = df_coinbase.pair.str.replace('-', '')
df_coinbase['time'] = pd.to_datetime(df_coinbase.date).dt.floor('S').dt.tz_convert(tz='Europe/London').dt.floor('s')
df_coinbase['size'] = df_coinbase['size'].astype('float') 
df_coinbase.total = df_coinbase.total.abs()
re_pair = re.compile('(?P<base>[a-z]+)(?P<quote>GBP|USDT|ETH|BTC|USDC|DAI|BUSD)', re.IGNORECASE)

df_coinbase[['base', 'quote']] = df_coinbase.pair.str.extract(re_pair, expand=True)

df_coinbase['exchange'] = 'coinbase-pro'
print({
    '# records': df_coinbase.shape[0],
    ### Total includes fee
    'price * size != total' : df_coinbase[(df_coinbase.price * df_coinbase.size - df_coinbase.total).abs() > 0.000001].shape[0],
    'price/fee/total unit != quote' : df_coinbase[df_coinbase['price/fee/total unit'] != df_coinbase.quote].shape[0],
    'size unit != base' : df_coinbase[df_coinbase['size unit'] != df_coinbase.base].shape[0],
    "'size', 'price', 'total', 'fee' < 0" : df_coinbase[['size', 'price', 'total', 'fee']].lt(0).sum().sum()

})
# base on the above observations, remove columns
df_coinbase.drop(['portfolio', 'trade id', 'price/fee/total unit', 'size unit'], inplace=True, axis=1)
df_coinbase['fee_curr'] = df_coinbase.quote
# df_coinbase['size'] = df_coinbase.size.abs()

df_coinbase = df_coinbase[['time', 'exchange', 'pair', 'base', 'quote', 'side', 'price', 'size', 'total', 'fee', 'fee_curr']]


# df_coinbase.shape, df_coinbase.dtypes,

df_coinbase


{'# records': 576, 'price * size != total': 576, 'price/fee/total unit != quote': 0, 'size unit != base': 0, "'size', 'price', 'total', 'fee' < 0": 0}


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
0,2021-09-24 00:12:51+01:00,coinbase-pro,DOTGBP,DOT,GBP,SELL,23.894,9.839000,233.917601,1.175465,GBP
1,2020-11-23 23:45:46+00:00,coinbase-pro,XRPGBP,XRP,GBP,BUY,0.455,10000.000000,4554.550000,4.550000,GBP
2,2020-11-24 15:10:43+00:00,coinbase-pro,XRPGBP,XRP,GBP,BUY,0.490,4959.000000,2432.339910,2.429910,GBP
3,2020-11-24 15:10:43+00:00,coinbase-pro,XRPGBP,XRP,GBP,BUY,0.490,5041.000000,2472.560090,2.470090,GBP
4,2020-11-24 15:48:12+00:00,coinbase-pro,XRPGBP,XRP,GBP,SELL,0.500,10010.000000,4999.995000,5.005000,GBP
...,...,...,...,...,...,...,...,...,...,...,...
571,2020-12-25 21:44:42+00:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,17920.000,0.943705,16924.714484,13.528948,GBP
572,2021-01-04 18:57:23+00:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,23000.000,0.845410,19463.883409,19.444439,GBP
573,2021-01-04 18:57:24+00:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,23000.000,0.240110,5528.060818,5.522538,GBP
574,2022-09-22 20:07:29+01:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.350,0.094000,1616.510113,9.641213,GBP


In [31]:
df_coinbase.fee_curr.unique()
df_coinbase[df_coinbase.fee_curr != df_coinbase.quote]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr


In [29]:
23.894	* 9.839000, 233.917601	+ 1.175465
# df_coinbase['tot'] = df_coinbase['price'] * df_coinbase['size']
# df_coinbase.head()
# round(df_coinbase['price'] * df_coinbase['size'] - df_coinbase['total'] + df_coinbase['fee'], 4)
# df_coinbase['date'].dt.tz_convert('Europe/London')

(235.093066, 235.093066)

### Load Kucoin Data

In [638]:
# import warnings

# with warnings.catch_warnings(record=True):
# warnings.simplefilter("always")
    # myexcelfile = pd.read_excel(easy_payfile, engine="openpyxl")

df_kucoin = pd.concat([
    pd.read_excel('./trading-data/kucoin/BillingHistory-2021/Spot Orders_Completed Trades20230131.xlsx', parse_dates=True),
    pd.read_excel('./trading-data/kucoin/BillingHistory-2022/Spot Orders_Completed Trades20230131.xlsx', parse_dates=True)
], axis=0)

df_kucoin.rename({'Symbol': 'pair', 
                    'Fee Currency': 'fee_curr',
                    'Avg. Filled Price' : 'price',
                    'Filled Amount': 'size',
                    'Filled Volume': 'total',
                    'Side': 'side',
                    'Fee': 'fee',
                    'Filled Time(UTC+08:00)': 'date',
                    }, inplace=True, axis='columns')
df_kucoin.pair = df_kucoin.pair.str.replace('-', '')
df_kucoin['time'] = pd.to_datetime(df_kucoin.date, infer_datetime_format=True, utc=True).dt.floor('S').dt.tz_convert(tz='Europe/London')
re_pair = re.compile('(?P<base>[a-z]+)(?P<quote>GBP|USDT|ETH|BTC|USDC|DAI|BUSD|UST)', re.IGNORECASE)

df_kucoin[['base', 'quote']] = df_kucoin.pair.str.extract(re_pair, expand=True)

df_kucoin['exchange'] = 'kucoin'
print({
    '# records': df_kucoin.shape[0],
    ### Total includes fee
    'size * price != total' : df_kucoin[(df_kucoin['price'] * df_kucoin['size'] - df_kucoin.total).abs() > 0.0001].astype(int).sum().sum(),
    'fee_curr != quote' : df_kucoin[df_kucoin['fee_curr'] != df_kucoin.quote].shape[0],
    # "'size', 'price', 'total', 'fee' < 0" : df_coinbase[['size', 'price', 'total', 'fee']].lt(0).sum().sum()

})
# base on the above observations, remove columns
# df_coinbase.drop(['portfolio', 'trade id', 'price/fee/total unit', 'size unit'], inplace=True, axis=1)
# df_coinbase['fee_curr'] = df_coinbase.quote
# df_coinbase['size'] = df_coinbase.size.abs()
df_kucoin['exchange'] = 'kucoin'
df_kucoin = df_kucoin[['time', 'exchange', 'pair', 'base', 'quote', 'side', 'price', 'size', 'total', 'fee', 'fee_curr']]

# df_kucoin.shape, 
df_kucoin


{'# records': 55, 'size * price != total': 0.0, 'fee_curr != quote': 0}


/home/grenada/miniconda3/envs/py311/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/grenada/miniconda3/envs/py311/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/tmp/ipykernel_4420/1967631420.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_kucoin['time'] = pd.to_datetime(df_kucoin.date, infer_datetime_format=True, utc=True).dt.floor('S').dt.tz_convert(tz='Europe/London')


,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr
0,2021-08-19 22:21:31+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,24.1864,5.000000e-02,1.209320,1.209320e-03,USDT
1,2021-08-20 01:02:28+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.7575,5.000000e-02,1.287875,1.287875e-03,USDT
2,2021-08-20 01:19:51+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.3844,1.000000e-02,0.253844,2.538440e-04,USDT
3,2021-08-20 01:28:19+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4126,1.000000e-02,0.254126,2.541260e-04,USDT
4,2021-08-20 01:28:25+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4067,1.000000e-02,0.254067,2.540670e-04,USDT
5,2021-08-20 01:28:30+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4116,1.000000e-02,0.254116,2.541160e-04,USDT
6,2021-08-20 01:28:36+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4175,1.000000e-02,0.254175,2.541750e-04,USDT
7,2021-08-20 01:28:42+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4060,1.000000e-02,0.254060,2.540600e-04,USDT
8,2021-08-20 01:28:48+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.4043,1.000000e-02,0.254043,2.540430e-04,USDT
9,2021-08-20 01:28:54+01:00,kucoin,DOTUSDT,DOT,USDT,SELL,25.3892,1.000000e-02,0.253892,2.538920e-04,USDT


In [40]:
# df_kucoin[df_kucoin.fee_curr != df_kucoin.quote]
# df_kucoin['date'].dt.tz_convert(tz='Europe/London')

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr


## Consolidate trades across exchanges

In [734]:
df_trades = pd.concat([df_binance, df_coinbase, df_kucoin], axis=0)
df_trades.time = df_trades.time.dt.tz_localize(None)
# 
# Drop total as it will be calculated inc fees
# df_trades.drop(['total'], axis=1, inplace=True)
df_trades.sort_values(by='time', ascending=True, inplace=True)
df_trades.reset_index(inplace=True, drop=True)
df_trades['ctotal'] = df_trades['size'] * df_trades.price

# calculate fee in quote currency
# df_trades['fee_q'] = np.where(
#     df_trades.fee_curr == df_trades.quote, df_trades.fee, 
#     np.where(df_trades.fee_curr == df_trades.base, df_trades.fee * df_trades.price, pd.NA)
#     )

df_trades['date'] = df_trades.time.dt.floor('d').dt.tz_localize(None)
# df_trades['ndate30p'] = (df_trades.date + timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
df_trades['ndate'] = df_trades.date.dt.strftime('%Y%m%d').astype(int)
df_trades['ndate30m'] = (df_trades.date - timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)

# df_trades.set_index('time', inplace=True, drop=True)
# df_trades['tridx'] = df_trades.index
# df_trades['calc_total'] = df_trades['size'] * df_trades.price
# df_trades['mtime'] = df_trades.time.dt.tz_localize(None).dt.floor(freq='T')
df_trades['ntime'] = df_trades.time.dt.strftime('%Y%m%d%H%M').astype(int)

df_trades

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,date,ndate,ndate30m,ntime
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,2020-09-15,20200915,20200816,202009151224
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,2020-09-15,20200915,20200816,202009151548
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,2020-09-15,20200915,20200816,202009151721
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,2020-09-17,20200917,20200818,202009170044
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,2020-09-17,20200917,20200818,202009170055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,2022-09-16,20220916,20220817,202209162320
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,2022-09-16,20220916,20220817,202209162320
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,2022-09-16,20220916,20220817,202209162320
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,2022-09-22,20220922,20220823,202209222007


In [640]:
df_trades[df_trades.total != df_trades.ctotal]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,202009151224
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,202009151548
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,202009151721
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,202009170044
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,202009170055
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27199,2022-09-16 23:13:17,binance,BTCGBP,BTC,GBP,BUY,17291.62,0.025310,437.650902,0.000000,BNB,437.650902,202209162313
27201,2022-09-16 23:14:08,binance,BTCGBP,BTC,GBP,BUY,17309.52,0.005120,88.624742,0.000000,BNB,88.624742,202209162314
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,202209162320
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,202209222007


## Calculate prices and fees in GBP

In [232]:

trade_n_times = sorted(df_trades.ntime.unique())
# trade_n_times

### Quote currency prices in GBP

-load minute data for quote currencies from an aggregator

In [642]:
### Load 

# def load_candles(filename):

def load_quote_hist(filename=None, ohlcv=None, ntimes=None):
  df = None
  if ohlcv is not None:
    df = pd.DataFrame(ohlcv, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
  else:
    df = pd.read_json(filename)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
  # filename = f'./freq-user-data/data/{exchange.lower()}/{pair}-1m.json'
  df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True).dt.tz_convert('Europe/London').dt.tz_localize(None)
  df['ntime'] = df.time.dt.strftime('%Y%m%d%H%M').astype(int)
  df.set_index('time', inplace=True, drop=True)
  
  df['ohl3'] = (df.open + df.high + df.low) / 3.
  df.drop(['open', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  if ntimes is not None: df = df[df.ntime.isin(ntimes)]
  # df.drop(['timestamp', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  # df['ohl3'] = (df['open'] + df['high'] + df['low']) / 3
  return df

qdf_eth_gbp  = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='ETH_GBP', exchange='binance'), ntimes=trade_n_times)
qdf_bnb_gbp  = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='BNB_GBP', exchange='binance'), ntimes=trade_n_times)
qdf_btc_gbp  = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='BTC_GBP', exchange='binance'), ntimes=trade_n_times)
qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)

# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# qdf_gbp_usdt = load_quote_hist('./freq-user-data/data/{exchange}/{pair}-1m.json'.format(pair='GBP_USDT', exchange='binance'), ntimes=trade_n_times)
# 'UST', 'BUSD', 'DAI', 'USDC


In [643]:
df_q = pd.concat([
    df['ohl3'] for df in [qdf_btc_gbp, qdf_eth_gbp, qdf_bnb_gbp, qdf_gbp_usdt]
], axis=1, keys=['BTC', 'ETH', 'BNB', 'USDT'])

df_q['ntime'] = df_q.index.strftime('%Y%m%d%H%M').astype(int)
df_q['USDT'] = df_q.USDT ** -1

In [644]:
# Check for missing historical data
df_trades[~df_trades.ntime.isin(df_q.ntime)]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime
556,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.00,3.120000,1402.876800,1.123200,GBP,1404.000000,202012250251
557,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.00,0.663350,297.970146,0.537313,GBP,298.507459,202012250251
558,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.18,7.015712,3152.648314,5.685000,GBP,3158.333314,202012250251
559,2020-12-25 02:51:57,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.21,4.000000,1797.598488,3.241512,GBP,1800.840000,202012250251
560,2020-12-25 02:52:16,coinbase-pro,ETHGBP,ETH,GBP,SELL,450.00,0.000938,0.421717,0.000338,GBP,0.422055,202012250252


In [645]:
# Manually get the missing historical klines from another exchange
import ccxt
ex = ccxt.coinbasepro()

df_append = pd.concat(
    [
        load_quote_hist(ohlcv=ohlcv) for ohlcv in 
        [ ex.fetch_ohlcv('ETH/GBP', '1m', limit=1, since=ex.parse8601(str(t))) for t in np.unique(df_trades[~df_trades.ntime.isin(df_q.ntime)].time.values.astype('<M8[m]').astype('<M8[s]')) ]
    ]
).rename({'ohl3': 'ETH'}, axis=1, inplace=False)

df_q = pd.concat([df_q, df_append])
df_q.sort_index(inplace=True)
df_q = df_q[~df_q.index.duplicated(keep='first')]


In [646]:
df_append

,ntime,ETH
time,,
2020-12-25 02:51:00,202012250251,450.140000
2020-12-25 02:52:00,202012250252,450.233333


In [647]:
# Check again for missing historical data
np.unique(df_trades[~df_trades.ntime.isin(df_q.ntime)].time.values.astype('<M8[m]')).astype('<M8[s]')

array([], dtype='datetime64[s]')

### Calculate quote price and fee in GBP

In [648]:
df_q

,BTC,ETH,BNB,USDT,ntime
time,,,,,
2020-09-15 12:24:00,8445.000000,295.00,22.740,0.779120,202009151224
2020-09-15 15:48:00,8400.026667,286.19,22.100,0.782228,202009151548
2020-09-15 17:21:00,8387.576667,283.84,22.042,0.780945,202009151721
2020-09-17 00:44:00,8506.130000,283.88,21.659,0.776096,202009170044
2020-09-17 00:55:00,8496.550000,283.74,21.636,0.776096,202009170055
...,...,...,...,...,...
2022-09-11 17:20:00,18726.283333,1532.25,254.700,0.863061,202209111720
2022-09-16 23:13:00,17301.206667,1262.92,240.200,0.876168,202209162313
2022-09-16 23:14:00,17291.713333,1261.62,240.200,0.875912,202209162314


In [735]:
# calculate price and fee in GBP

for row in df_trades.itertuples():
  # base, quote, fee_curr, price, fee = row.base, row.quote, row.fee_curr, row.price, row.fee
  # print(f'Processing row: {row}')
  quote = row.quote
  if quote in ['UST', 'BUSD', 'DAI', 'USDC'] : quote = 'USDT'
  gprice = pd.NA
  qprice = pd.NA
  if row.base != 'GBP' and row.quote != 'GBP':
    qprice = df_q.loc[row.time.replace(second=0), quote]

  if row.quote == 'GBP':
    gprice = row.price
  elif row.base == 'GBP':
    gprice = row.price ** -1
  else:
    gprice = row.price * qprice

  gfee = pd.NA
  if row.fee_curr == 'GBP': 
    gfee = row.fee
  elif row.fee_curr == row.quote:
    qprice = df_q.loc[row.time.replace(second=0), quote]
    gfee = row.fee * qprice
  elif row.fee_curr == row.base:
    gfee = row.fee * gprice
  else:
    gfee = row.fee * df_q.loc[row.time.replace(second=0), row.fee_curr]

  df_trades.at[row.Index,'gfee'] = round(gfee,2)
  df_trades.at[row.Index,'gprice'] = gprice
  df_trades.at[row.Index,'qprice'] = qprice

df_trades

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,date,ndate,ndate30m,ntime,gfee,gprice,qprice
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,2020-09-15,20200915,20200816,202009151224,0.42,8420.00,NaN
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,2020-09-15,20200915,20200816,202009151548,0.42,8400.00,NaN
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,2020-09-15,20200915,20200816,202009151721,0.42,8375.00,NaN
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,2020-09-17,20200917,20200818,202009170044,0.42,8480.00,NaN
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,2020-09-17,20200917,20200818,202009170055,0.70,282.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,2022-09-16,20220916,20220817,202209162320,0.00,17295.79,NaN
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,2022-09-16,20220916,20220817,202209162320,0.00,17300.00,NaN
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,2022-09-16,20220916,20220817,202209162320,0.00,17296.68,NaN
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,2022-09-22,20220922,20220823,202209222007,9.64,17094.35,NaN


In [650]:
# Check for unpopulated gfee and gprice (fee and price in GBP)
df_trades[df_trades.gprice.isna() | df_trades.gfee.isna()]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime,gfee,gprice


In [651]:
# calculate amount and total in GBP
# df_trades['gamount'] = (df_trades['size'] * df_trades.gprice).round(2)
# df_trades['gtotal'] = df_trades['gamount'] + df_trades['gfee'] * np.where(df_trades.side == 'BUY', 1, -1) * np.where(df_trades.base == 'GBP', -1, 1)
# df_trades['gfprice'] = df_trades['gtotal'] / df_trades['size']
# df_trades[(df_trades.fee_curr == df_trades.quote) ].fee_curr.unique()
# df_trades.fee_curr.unique()
df_trades

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime,gfee,gprice
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,202009151224,0.42,8420.00
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,202009151548,0.42,8400.00
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,202009151721,0.42,8375.00
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,202009170044,0.42,8480.00
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,202009170055,0.70,282.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,202209162320,0.00,17295.79
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,202209162320,0.00,17300.00
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,202209162320,0.00,17296.68
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,202209222007,9.64,17094.35


In [746]:
# check different cases are sane
df_trades[df_trades.fee_curr == df_trades.base]
df_trades[df_trades.fee_curr == df_trades.quote]
df_trades[(df_trades.base == 'GBP') & (df_trades.gfee != 0)]
df_trades[df_trades.qprice.isna() & (df_trades.quote != 'GBP') & (df_trades.base != 'GBP')]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,date,ndate,ndate30m,ntime,gfee,gprice,qprice


## Create Acc / Disposal dataframe

In [718]:
# df_trades[acc_cols][(df_trades.quote == 'GBP') & (df_trades.side == 'BUY')].assign(tpy='ACC',src='TRADE',gprice=lambda x:x['price']).rename({'base': 'asset', 'size': 'qty'}, axis=1).drop(['quote'], axis=1),



In [744]:
# acc_cols = ['time', 'pair','base', 'quote', 'size', 'price', 'ctotal', 'qprice', 'gprice', 'gfee']
# acc_cross_cols = ['time', 'pair', 'quote', 'size', 'price', 'ctotal', 'qprice', 'gprice', 'gfee']
df_acc_dis = pd.concat([
df_trades[(df_trades.quote == 'GBP') & (df_trades.side == 'BUY' )].assign(tpy='ACC',src='TRADE',asset=lambda x:x['base'],aprice=lambda x:x['price'],aqty=lambda x:x['size']),
df_trades[(df_trades.quote == 'GBP') & (df_trades.side == 'SELL')].assign(tpy='DIS',src='TRADE',asset=lambda x:x['base'],aprice=lambda x:x['price'],aqty=lambda x:x['size']),
df_trades[(df_trades.base == 'GBP') & (df_trades.side == 'BUY')].assign(tpy='DIS',src='TRADE',asset=lambda x:x['quote'],aprice=lambda x:x['price']**-1,aqty=lambda x:x['ctotal']),
df_trades[(df_trades.base == 'GBP') & (df_trades.side == 'SELL')].assign(tpy='ACC',src='TRADE',asset=lambda x:x['quote'],aprice=lambda x:x['price']**-1,aqty=lambda x:x['ctotal']),
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'BUY')].assign(tpy='ACC',src='TRADE',asset=lambda x:x['base'],aprice=lambda x:x['gprice'],aqty=lambda x:x['size']),
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'SELL')].assign(tpy='DIS',src='TRADE',asset=lambda x:x['base'],aprice=lambda x:x['gprice'],aqty=lambda x:x['size']),
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'BUY')].assign(tpy='DIS',src='TRADE',asset=lambda x:x['quote'],aprice=lambda x:x.qprice,aqty=lambda x:x['ctotal']),
df_trades[(df_trades.base != 'GBP') & (df_trades.quote != 'GBP') & (df_trades.side == 'SELL')].assign(tpy='ACC',src='TRADE',asset=lambda x:x['quote'],aprice=lambda x:x.qprice,aqty=lambda x:x['ctotal'])
],axis=0, ignore_index=True)

#add manually busd received as rewards
df_acc_dis.loc[len(df_acc_dis)] = {'time':pd.to_datetime('2021-01-07 08:44:44'), 'asset':'BUSD','tpy':'ACC','qty':7, 'src': 'DISTR'}
df_acc_dis.loc[len(df_acc_dis)] = {'time':pd.to_datetime('2020-12-30 03:33:06'), 'asset':'BUSD','tpy':'ACC','qty':20, 'src': 'DISTR'}

# df_acc_dis['ntime'] = df_acc_dis.time.dt.strftime('%Y%m%d%H%M').astype(int)
df_acc_dis['date'] = df_acc_dis.time.dt.floor('d').dt.tz_localize(None)
# df_acc_dis['ndate30p'] = (df_acc_dis.date + timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
df_acc_dis['ndate'] = df_acc_dis.date.dt.strftime('%Y%m%d').astype(int)
df_acc_dis['ndate30m'] = (df_acc_dis.date - timedelta(days=30)).dt.strftime('%Y%m%d').astype(int)
# df_acc_dis = df_acc_dis[['time', 'ntime', 'date',
       # 'ndate', 'ndate30m', 'pair', 'asset', 'src', 'tpy', 'qty', 'price', 'ctotal', 'qprice', 'gprice', 'gfee']]
# df_acc_dis['qty_left'] =df_acc_dis.qty
df_acc_dis['bnb_qty'] = 0.0
df_acc_dis = df_acc_dis.sort_values(['time', 'tpy'])
df_acc_dis.reset_index()
# df_acc_dis.set_index('time',inplace=True)
df_acc_dis

,time,exchange,pair,base,quote,side,price,size,total,fee,...,ntime,gfee,gprice,qprice,tpy,src,asset,aprice,aqty,bnb_qty
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,...,2.020092e+11,0.42,8420.00,NaN,ACC,TRADE,BTC,8420.00,0.010000,0.0
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,...,2.020092e+11,0.42,8400.00,NaN,ACC,TRADE,BTC,8400.00,0.010000,0.0
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,...,2.020092e+11,0.42,8375.00,NaN,ACC,TRADE,BTC,8375.00,0.010000,0.0
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,...,2.020092e+11,0.42,8480.00,NaN,ACC,TRADE,BTC,8480.00,0.010000,0.0
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,...,2.020092e+11,0.70,282.00,NaN,ACC,TRADE,ETH,282.00,0.500000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,...,2.022092e+11,0.00,17295.79,NaN,ACC,TRADE,BTC,17295.79,0.474600,0.0
1258,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,...,2.022092e+11,0.00,17300.00,NaN,ACC,TRADE,BTC,17300.00,0.061490,0.0
1259,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,...,2.022092e+11,0.00,17296.68,NaN,ACC,TRADE,BTC,17296.68,0.009330,0.0
1260,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,...,2.022092e+11,9.64,17094.35,NaN,ACC,TRADE,BTC,17094.35,0.094000,0.0


In [654]:
df_acc_dis.time

0      2020-09-15 12:24:15
1      2020-09-15 15:48:15
2      2020-09-15 17:21:13
3      2020-09-17 00:44:00
4      2020-09-17 00:55:05
               ...        
1257   2022-09-16 23:20:12
1258   2022-09-16 23:20:12
1259   2022-09-16 23:20:12
1260   2022-09-22 20:07:29
1261   2022-09-22 20:07:29
Name: time, Length: 51302, dtype: datetime64[ns]

In [655]:
df_acc_dis[df_acc_dis.gprice.isna()],
df_acc_dis[df_acc_dis.qty.isna()],
df_acc_dis[df_acc_dis.gfee.isna()]

,time,ntime,date,ndate,ndate30m,asset,src,tpy,qty,gprice,gfee,bnb_qty
51301,2020-12-30 03:33:06,202012300333,2020-12-30,20201230,20201130,BUSD,DISTR,ACC,20.0,NaN,NaN,0.0
51300,2021-01-07 08:44:44,202101070844,2021-01-07,20210107,20201208,BUSD,DISTR,ACC,7.0,NaN,NaN,0.0


## Calculate pools and bed-n-breakfast rule

In [719]:
# Bed-n-breakfast rule
for row in df_acc_dis[df_acc_dis.tpy=='DIS'].itertuples():
    #bnb rule
    df_bnb = df_acc_dis[(df_acc_dis.tpy=='ACC') & (df_acc_dis.src=='DISTR') & (df_acc_dis.asset==row.asset) & (df_acc_dis.ndate > row.ndate) & (df_acc_dis.ndate30m <= row.ndate) & (acc_row.qty > acc_row.bnb_qty)]
    if len(df_bnb) > 0:
        
        tqty, tcost, tfees = 0.,0.,0.
        for acc_row in df_bnb.itertuples():
            dqty = row.qty - tqty
            aqty = min(dqty, acc_row.qty - acc_row.bnb_qty)
            df_acc_dis.at[acc_row.Index,'bnb_qty'] = acc_row.bnb_qty + aqty

            tqty += aqty
            tcost = round(tcost + aqty*acc_row.gprice,2)
            tfees = round(tfees + acc_row.gfee * aqty / acc_row.qty,2)


            if dqty <= 0: break;

        # df_acc_dis.at[row.Index,'qty_left'] = row.qty_left - tqty
        df_acc_dis.at[row.Index,'bnb_qty'] = tqty
        df_acc_dis.at[row.Index,'bnb_cost'] = round(tcost,2)
        df_acc_dis.at[row.Index,'bnb_fees'] = round(tfees + row.gfee*tqty/row.qty,2)
        # print(len(df_bnb),row,df_bnb)
  

df_acc_dis[(df_acc_dis.bnb_qty!=0)]

,time,ntime,date,ndate,ndate30m,pair,asset,src,tpy,qty,price,qprice,gprice,gfee,bnb_qty


In [748]:
# Pools

pools = {}
for row in df_acc_dis.itertuples():
    if not row.asset in pools:
        pools[row.asset] = {'pqty': 0.0, 'pcost': 0.0}

    passet = pools[row.asset]
    if row.tpy == 'ACC':
        aqty = row.aqty - row.bnb_qty
        passet['pqty'] += aqty
        # passet['pfees'] += row.gfee * aqty/row.qty
        passet['pcost'] += round(aqty * row.gprice + row.gfee * aqty/row.aqty,2)

        df_acc_dis.at[row.Index, 'pqty'] = passet['pqty']
        # df_acc_dis.at[row.Index, 'pfees'] = passet['pfees']
        df_acc_dis.at[row.Index, 'pcost'] = passet['pcost']
    else:
        if passet['pqty'] == 0:
            print("passet['pqty']=0\n", row, "\n", passet)
        aqty = row.aqty - row.bnb_qty
        cost = passet['pcost'] * aqty/passet['pqty']
        passet['pqty'] -= aqty
        passet['pcost'] -= cost
        proceeds = aqty * row.gprice
        dis_fees = aqty/row.aqty * row.gfee

        df_acc_dis.at[row.Index, 'gain'] = round(proceeds - cost - dis_fees, 2) 

        df_acc_dis.at[row.Index, 'pqty'] = passet['pqty']
        # df_acc_dis.at[row.Index, 'pfees'] = passet['pfees']
        df_acc_dis.at[row.Index, 'pcost'] = round(passet['pcost'],2)

    
pools

{'BTC': {'pqty': 11.018487472796163, 'pcost': 333269.9813892366},
 'ETH': {'pqty': 3.2541014899999112, 'pcost': 4065.10418510374},
 'XRP': {'pqty': 55.000000000002274, 'pcost': 10.189012147111328},
 'ALGO': {'pqty': 44557.28, 'pcost': 60496.41815590197},
 'ADA': {'pqty': 46327.35999999995, 'pcost': 64774.30114646493},
 'LINK': {'pqty': 17.242000000002342, 'pcost': 364.27508701971146},
 'BUSD': {'pqty': nan, 'pcost': nan},
 'BNB': {'pqty': 208.8019999999999, 'pcost': 52828.35156661443},
 'DOT': {'pqty': 25.547899070006352, 'pcost': 906.9262312241176},
 'USDT': {'pqty': -28191.49150923898, 'pcost': -1616125.870525278},
 'XLM': {'pqty': 282.39999999990687, 'pcost': 81.0606216631927},
 'LTC': {'pqty': 0.18800000000018133, 'pcost': 36.83095466670178},
 'AVAX': {'pqty': 34.91999999999887, 'pcost': 2832.820780480937},
 'AAVE': {'pqty': 0.05000000000000426, 'pcost': 17.972679073795007},
 'EGLD': {'pqty': 0.5999999999998558, 'pcost': 117.876367251165},
 'ATOM': {'pqty': 52.45999999999339, 'pcos

In [703]:
df_acc_dis.groupby('asset').count()

,time,ntime,date,ndate,ndate30m,pair,src,tpy,qty,price,qprice,gprice,gfee,bnb_qty,pqty,pcost,gain
asset,,,,,,,,,,,,,,,,,
AAVE,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,17
ADA,2434,2434,2434,2434,2434,2434,2434,2434,2434,2434,2429,2429,2434,2434,2434,1208,651
ALGO,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,5518,3510
ATOM,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,2999,1891
AVAX,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,1017
BNB,280,280,280,280,280,280,280,280,280,280,274,274,280,280,280,149,65
BTC,11303,11303,11303,11303,11303,11303,11303,11303,11303,11303,8660,8660,11303,11303,11303,0,0
BUSD,242,242,242,242,242,240,242,242,242,240,240,240,240,242,242,0,0
CAKE,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,62


In [668]:
df_acc_dis[df_acc_dis.asset=='REN']

,time,ntime,date,ndate,ndate30m,asset,src,tpy,qty,gprice,gfee,bnb_qty,pqty,pcost,gain
10930,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,1014.0,0.710920,0.72,0.0,1014.0,721.59,NaN
10931,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,224.0,0.711429,0.16,0.0,1238.0,881.11,NaN
10932,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,306.0,0.710339,0.22,0.0,1544.0,1098.69,NaN
10933,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,12.0,0.711792,0.01,0.0,1556.0,1107.24,NaN
10934,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,225.0,0.710920,0.16,0.0,1781.0,1267.36,NaN
10935,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,2044.0,0.710412,1.45,0.0,3825.0,2720.89,NaN
10936,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,1375.0,0.710412,0.98,0.0,5200.0,3698.69,NaN
10937,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,607.0,0.711065,0.43,0.0,5807.0,4130.74,NaN
10938,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,1532.0,0.710993,1.09,0.0,7339.0,5221.07,NaN
10939,2021-11-03 12:49:04,202111031249,2021-11-03,20211103,20211004,REN,TRADE,ACC,470.0,0.711719,0.33,0.0,7809.0,5555.91,NaN


In [628]:
11695.0 - 1032.0
1032.0 * 0.697869 - 8323.004092 * 1032.0 / 11695.0 - 0.72

-14.964700588627553

In [630]:
df_acc_dis[df_acc_dis.asset=='USDT']

,time,ntime,date,ndate,ndate30m,asset,src,tpy,qty,gprice,gfee,bnb_qty,pqty,pcost,gain
3089,2021-01-16 19:43:43,202101161943,2021-01-16,20210116,20201217,USDT,TRADE,ACC,185.83000,0.737953,0.00,0.0,1.858300e+02,1.371338e+02,NaN
3090,2021-01-16 19:43:43,202101161943,2021-01-16,20210116,20201217,USDT,TRADE,ACC,0.01000,0.737898,0.00,0.0,1.858400e+02,1.371412e+02,NaN
3091,2021-01-16 19:43:43,202101161943,2021-01-16,20210116,20201217,USDT,TRADE,ACC,140.00000,0.737953,0.10,0.0,3.258400e+02,2.405546e+02,NaN
3092,2021-01-16 19:43:43,202101161943,2021-01-16,20210116,20201217,USDT,TRADE,ACC,140.00000,0.737898,0.10,0.0,4.658400e+02,3.439604e+02,NaN
3093,2021-01-16 19:43:48,202101161943,2021-01-16,20210116,20201217,USDT,TRADE,ACC,54.26000,0.737953,0.00,0.0,5.201000e+02,3.840017e+02,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,2022-01-21 11:34:45,202201211134,2022-01-21,20220121,20211222,USDT,TRADE,DIS,2226.06000,0.737700,0.16,0.0,-1.396653e+09,-1.322383e+13,-2.107521e+07
36460,2022-09-11 17:20:07,202209111720,2022-09-11,20220911,20220812,USDT,TRADE,DIS,0.19599,18723.598964,0.00,0.0,-1.396653e+09,-1.322383e+13,1.813959e+03
36461,2022-09-11 17:20:07,202209111720,2022-09-11,20220911,20220812,USDT,TRADE,DIS,0.50038,18723.607595,0.00,0.0,-1.396653e+09,-1.322383e+13,4.631205e+03
36462,2022-09-11 17:20:07,202209111720,2022-09-11,20220911,20220812,USDT,TRADE,DIS,0.12000,18723.495397,0.00,0.0,-1.396653e+09,-1.322383e+13,1.110632e+03


In [749]:
df_acc_dis.loc[df_acc_dis.ndate<=20210405].assign(cumgain=lambda x: x['gain'].cumsum().round(2)).to_csv('./temp/df_acc_dis-2021-04-05.csv')
df_acc_dis.loc[(df_acc_dis.ndate<=20210405) & (df_acc_dis.asset=='BTC')].assign(cumgain=lambda x: x['gain'].cumsum().round(2)).to_csv('./temp/df_acc_dis-BTC-2021-04-05.csv')
# df_acc_dis.loc[df_acc_dis.ndate<=20220405].gain.sum()

In [671]:
df_trades[df_trades.time=='2020-12-08 23:19:09']

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime,gfee,gprice
358,2020-12-08 23:19:09,coinbase-pro,XRPBTC,XRP,BTC,BUY,0.00003,2642.0,0.080503,0.000080,BTC,0.080422,202012082319,1.11,0.419468
359,2020-12-08 23:19:09,coinbase-pro,XRPBTC,XRP,BTC,BUY,0.00003,20654.0,0.629336,0.000629,BTC,0.628708,202012082319,8.66,0.419468


In [17]:
def load_candles(pair):
  filename = f'./freq-user-data/data/binance/{pair}-1m.json'
  df = pd.read_json(filename)
  df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
  # df.columns = ['timestamp', 'open']
  df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True).dt.tz_convert('Europe/London').dt.tz_localize(None)
  # df = df[(df['date']>=date(2020,4,6)) & (df['date']<=date(2021,4,5))] 
  # df['timestamp'] = df.timestamp.astype('datetime64[m]')
  # df.set_index(pd.DatetimeIndex(df["time"]), inplace=True, drop=True)
  # df = df['2020-04-06' : '2021-04-05']
  df['ohl3'] = (df.open + df.high + df.low) / 3.
  # df.drop(['timestamp', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
  # df['ohl3'] = (df['open'] + df['high'] + df['low']) / 3
  return df

df_btc = load_candles('BTC_GBP')
df_eth = load_candles('ETH_GBP')
df_usdt = load_candles('GBP_USDT')
df_usdt = pd.DataFrame({'time': df_usdt.time, 'ohl3': df_usdt.ohl3**(-1)})

# kline_pairs = [f.split('.')[0].split('-')[0] for f in listdir(datapath) if isfile(join(datapath, f))]
# klines = {pair: load_candles(f'./data/{pair}-1m.json') for pair in kline_pairs}
# klines


In [570]:
df_trades[(df_trades.base=='USDT') | ()]

,time,exchange,pair,base,quote,side,price,size,total,fee,fee_curr,ctotal,ntime,gfee,gprice
0,2020-09-15 12:24:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8420.00,0.010000,84.621000,0.421000,GBP,84.200000,202009151224,0.42,8420.00
1,2020-09-15 15:48:15,coinbase-pro,BTCGBP,BTC,GBP,BUY,8400.00,0.010000,84.420000,0.420000,GBP,84.000000,202009151548,0.42,8400.00
2,2020-09-15 17:21:13,coinbase-pro,BTCGBP,BTC,GBP,BUY,8375.00,0.010000,84.168750,0.418750,GBP,83.750000,202009151721,0.42,8375.00
3,2020-09-17 00:44:00,coinbase-pro,BTCGBP,BTC,GBP,BUY,8480.00,0.010000,85.224000,0.424000,GBP,84.800000,202009170044,0.42,8480.00
4,2020-09-17 00:55:05,coinbase-pro,ETHGBP,ETH,GBP,BUY,282.00,0.500000,141.705000,0.705000,GBP,141.000000,202009170055,0.70,282.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17295.79,0.474600,8208.581934,0.000000,BNB,8208.581934,202209162320,0.00,17295.79
27205,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17300.00,0.061490,1063.777000,0.000000,BNB,1063.777000,202209162320,0.00,17300.00
27206,2022-09-16 23:20:12,binance,BTCGBP,BTC,GBP,BUY,17296.68,0.009330,161.378024,0.000000,BNB,161.378024,202209162320,0.00,17296.68
27207,2022-09-22 20:07:29,coinbase-pro,BTCGBP,BTC,GBP,BUY,17094.35,0.094000,1616.510113,9.641213,GBP,1606.868900,202209222007,9.64,17094.35
